## Import required libraries

If <b>geemap</b> and <b>ee</b> are not installed, activate the following two lines:

In [10]:
# ! conda install -c conda-forge earthengine-api
# ! conda install -c conda-forge geemap

In [6]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geemap
import os
from IPython.display import HTML
import time

ee.Initialize()

html_data = """<img src='https://scontent.fskg1-2.fna.fbcdn.net/v/t1.15752-9/198303885_147013840714784_2198338795950191827_n.png?_nc_cat=105&ccb=1-3&_nc_sid=ae9488&_nc_ohc=f4300tM9WkwAX-XVA7q&_nc_ht=scontent.fskg1-2.fna&oh=f286b959f8a4ac4b66b80ab08ee14f63&oe=60C874D0'/><br/>"""
html = HTML(data=html_data)

## Define the area of study

The instructions to define the study area are:
1) Visit the website:
    http://geojson.io/#map=2/20.0/0.0

2) Select the area using the rectangle button (1st arrow in the image below)

3) Copy the GeoJSON string at the right of the window (2nd arrow in the image below)

4) Paste it at the "geoJSON" variable --> two cells bellow

The figure bellow shows the two actions of the step (1)

In [7]:
html

## Paste the GeoJSON string below

In [8]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              22.91147232055664,
              40.72293316385307
            ],
            [
              22.925033569335938,
              40.72293316385307
            ],
            [
              22.925033569335938,
              40.73412061435751
            ],
            [
              22.91147232055664,
              40.73412061435751
            ],
            [
              22.91147232055664,
              40.72293316385307
            ]
          ]
        ]
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

## Function "Sentinel 1" to retrieve σ$^0$ between two specific dates and export to .tiff

In [21]:
"""

The Google Earth Engine database of Sentinel 1 is containing image collection. 
You can retrieve all the images between two specific dates

date_start = 'yyyy/mm/dd', the starting date of the period of image collection
date_end = 'yyyy/mm/dd', the ending date of the period of image collection

export = True/False, Export all the images of the image collection. Default = True
if export is True:
    export_band = 'VV'/'VH', Select the band of the image collection. Default = 'VV'
    export_scale = float, Depending on the size of the area. The larger areas the lower the scale is. Default = 1
    
Timeseries = True/False, Print the timeseries of 'VV' and 'VH' and the inc angle between given dates. Default = True

"""

def sentinel_1(date_start, date_stop, area, export = True, export_band = 'VV', export_scale = 1, timeseries = True):
    if timeseries:
        dataset = (ee.ImageCollection('COPERNICUS/S1_GRD')
                            .filterBounds(area)
                            .filterDate(ee.Date(date_start),ee.Date(date_stop))
                            .filter(ee.Filter.eq('instrumentMode', 'IW'))
                            .sort('system:time_start'))

        acq_times = dataset.aggregate_array('system:time_start').getInfo()
        avalaible_dates = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
        im_list = dataset.toList(dataset.size())
        _list = ee.List([])

        def reduce_dataset_region(image, list):
            local_image = image.reduceRegion(
                reducer=ee.Reducer.median(),
                geometry=area,
                scale=10)
            return ee.List(list).add(local_image)

        reduced_dataset = dataset.iterate(reduce_dataset_region, _list)
        reduced_dataset_dict = reduced_dataset.getInfo()

        result = pd.DataFrame(reduced_dataset_dict)
        result.index = avalaible_dates
        result = result.dropna()
        print(result)

    if export:
        dataset_exp = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                        .filterBounds(area)
                        .filterDate(ee.Date(date_start),ee.Date(date_stop))
                        .filter(ee.Filter.eq('instrumentMode', 'IW'))
                        .sort('system:time_start')
                        .select(export_band))

        out_dir_col = os.path.join(os.path.expanduser('~'), 'Downloads')
        geemap.ee_export_image_collection(dataset_exp, region=area, scale=1, out_dir=out_dir_col)

## Input starting and ending date and run the function

In [22]:
date_start = '2020-01-01'
date_stop = '2020-01-03'

sentinel_1(date_start, date_stop, aoi, export = False, export_scale = 0.5, timeseries = True)

                 VH        VV      angle
01/01/20 -14.628168 -7.311737  31.042724
01/02/20 -15.060497 -8.063595  41.837157
01/02/20 -14.374747 -8.185926  38.153076
